In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import time
import sys
from pixell import enmap, enplot, reproject, utils, curvedsky 
from matplotlib import cm
from scipy.optimize import curve_fit
from scipy.linalg import sqrtm
from scipy import constants
from scipy import stats as sci_stats  
import nawrapper as nw
import math
import csv 

path = "/home/kw4043/project/Planck_cib_maps/"
path_beam = "/home/kw4043/project/Planck_cib_maps/beams/BeamWf_HFI_R3/"

path_HI_857_40 = "/home/kw4043/project/Planck_cib_maps/HI_maps/4.0e+20_gp40_857/"

##I'm gonna be using path_beam for the mcm's as well to separate from the previous notebooks

def eshow(x,**kwargs): 
    ''' Define a function to help us plot the maps neatly '''
    plots = enplot.get_plots(x, **kwargs)
    enplot.show(plots, method = "ipython")
    
def shrink_mask(mask, r):
    """Shrink the True part of boolean mask "mask" by a distance of r radians"""
    return mask.distance_transform(rmax=r) >= r

def coth(x):
    "function returns value for hyperbolic cotangent"
    return 1/np.tanh(x)

In [ ]:
%run Spectra_Results_For_Chi_Squared.ipynb

#relevant variables are CB_857_full_40 for the full ACT Y map, CB_857_cib_40 for the cib deprojected ACT Y map
#var_857_full_40 is the variance for the full ACT Y map, and var_857_cib_40 is the variance for the cib deprojected ACT Y map

In [ ]:
def template_indices_finder(CB_spectrum,template):
    #assumes that CB_spectrum is produced through nawrapper and that the template is a 2D array with [ells,power] as the indices
    template_ells = []
    
    for x in np.round(CB_spectrum['ell']):
        intermediate = np.where(template[0] == x)[0]
        template_ells.append(intermediate)
        
    template_ells = np.concatenate(template_ells)
    return template_ells   

In [ ]:
cib_SZ_template_array = path + 'SZ_template.txt'
cib_SZ_template_array = np.loadtxt(cib_SZ_template_array, unpack = True)
dell = (cib_SZ_template_array[0]*(cib_SZ_template_array[0] + 1))/(2*np.pi)
cib_SZ_template_array[1] /= dell

In [ ]:
total_template_ells = template_indices_finder(CB_857_full_40, cib_SZ_template_array)
print(total_template_ells)

In [ ]:
CB_857_full_40['ell']

In [ ]:
def chi_values_finder(CB_spectrum, possible_amplitudes_array, template, corresponding_template_ell_indices, variance, lmin, lmax):
    #I need to think more about units with this function. The amplitude can absorb the 1e12 factor, but it can't do the same with dell vs ell
    #This returns an array of chi_squared values, one per each possible template amplitude
    chi_values = []
    
    ell_mask = (CB_spectrum['ell'] >= lmin) & (CB_spectrum['ell'] <= lmax)
    
    for x in possible_amplitudes_array:
        Amp = x
        intermediate = ((CB_spectrum['TT'] - Amp*template[1][corresponding_template_ell_indices])**2)/(variance)
        chi_values.append(sum(intermediate[ell_mask]))
    
    return chi_values, np.sum(ell_mask)

In [ ]:
possible_amplitudes = np.linspace(1e-10,1e-8,1000)
chi_values, dof = chi_values_finder(CB_857_full_40, possible_amplitudes, cib_SZ_template_array, total_template_ells, var_857_full_40, 50, 1500)

In [ ]:
plt.plot(possible_amplitudes,chi_values)
plt.xscale('log')
plt.yscale('log')

In [ ]:
def minimized_chi_squared(chi_values_array, possible_amplitudes_array):
    #This function takes the long array of chi_squared values, one per each possible template amplitude, and finds the minimum chi_value
    #returning the corresponding minimized amplitude. It also returns a plot to show this minimization
    chi_values = np.array(chi_values_array)
    chi_min_index = np.where(chi_values == np.amin(chi_values))[0][0]
    minimized_amplitude = possible_amplitudes_array[chi_min_index]
    
    return minimized_amplitude, chi_values[chi_min_index]

In [ ]:
min_amp, min_chi = minimized_chi_squared(chi_values, possible_amplitudes)
print(min_amp, min_chi)
reduced_chi = min_chi / dof
print(reduced_chi)

In [ ]:
from scipy import stats
print(dof)
1 - stats.chi2.cdf(min_chi, dof)

In [ ]:
fig, ax  = plt.subplots(dpi=300)
ax.plot(cib_SZ_template_array[0],(min_amp)*cib_SZ_template_array[1])
ax.errorbar(CB_857_full_40['ell'],CB_857_full_40['TT'], yerr = var_857_full_40**0.5, ls = 'none', ms = 200)
ax.set_yscale('log')
ax.set_xlim(0,5000)
ax.set_ylim(5e-16,2e-14)

In [ ]:
#I believe this cell doesnt do anything useful and not redundant, but I'll check
#np.amin(chi_values)

#chi_min = np.where(chi_values == np.amin(chi_values))
#print(chi_min)

#chi_values[0]